![imagen](../../images/ejercicios.png)

In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager    

from bs4 import BeautifulSoup as bs
import time
import pandas as pd

from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [2]:
# opciones del webdriver

opciones=Options()

opciones.add_experimental_option('excludeSwitches', ['enable-automation'])
opciones.add_experimental_option('useAutomationExtension', False)

opciones.headless=False            

# Ejercicio 1 

Crea un dataframe con las novedades de mercadona

https://tienda.mercadona.es/

In [3]:
url='https://tienda.mercadona.es/'

In [4]:
PATH=ChromeDriverManager().install()  

[WDM] - 

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 96.0.4664
[WDM] - Get LATEST driver version for 96.0.4664
[WDM] - Driver [/Users/iudh/.wdm/drivers/chromedriver/mac64/96.0.4664.45/chromedriver] found in cache


In [5]:
driver=webdriver.Chrome(PATH, options=opciones)
driver.get(url)

time.sleep(1)

In [6]:
# cookies
driver.find_element_by_xpath('//*[@id="root"]/div[1]/div/div/button[2]').click()
time.sleep(1)

# codigo postal
driver.find_element_by_xpath('//*[@id="root"]/div[4]/div/div[2]/div/form/div/input').send_keys('28004')
time.sleep(1)
driver.find_element_by_xpath('//*[@id="root"]/div[4]/div/div[2]/div/form/button').click()


# espera a cargar 3 secs
time.sleep(3)

In [7]:
# parseo con bs4
soup=bs(driver.page_source, 'html.parser')

# productos
productos=soup.find_all('div', class_='product-cell__info')

# descripcion
desc=[p.find('h4').text for p in productos]

# formato
formato=[p.find('div', class_='product-format product-format__size--cell').text for p in productos]

# precio
precio=[p.find('p').text for p in productos]

# apagar driver
driver.quit()

In [8]:
# dataframe
columnas=['desc', 'formato', 'precio']

df=pd.DataFrame(columns=columnas)

df['desc']=desc
df['formato']=formato
df['precio']=precio

df.head()

,desc,formato,precio
0,Chocolate blanco Hacendado fussion relleno con...,Tableta 100 g,"0,80 €"
1,Recarga jabón de manos dermoprotector Deliplus...,Paquete 1 L,"1,10 €"
2,Paté de Jabalí con boletus Hacendado,Tarro 120 g,"1,50 €"
3,Paté de cerdo con cebolla caramelizada Argal,Lata 200 g,"1,50 €"
4,Jamón de cebo de campo ibérico 50% raza ibéric...,Pieza 7 kg,"149,00 €"


In [9]:
df['precio']=df['precio'].apply(lambda x: float(x.split()[0].replace(',', '.')))

df.head()

,desc,formato,precio
0,Chocolate blanco Hacendado fussion relleno con...,Tableta 100 g,0.8
1,Recarga jabón de manos dermoprotector Deliplus...,Paquete 1 L,1.1
2,Paté de Jabalí con boletus Hacendado,Tarro 120 g,1.5
3,Paté de cerdo con cebolla caramelizada Argal,Lata 200 g,1.5
4,Jamón de cebo de campo ibérico 50% raza ibéric...,Pieza 7 kg,149.0


# Ejercicio 2

Crea un dataframe con los juguetes con mayor descuento para black friday en Amazon.

https://www.amazon.es/

In [10]:
url='https://www.amazon.es/'

In [11]:
driver=webdriver.Chrome(PATH, options=opciones)
driver.get(url)

time.sleep(1)

In [12]:
# cookies
driver.find_element_by_xpath('//*[@id="sp-cc-accept"]').click()


# b-friday
driver.find_element_by_xpath('//*[@id="nav-xshop"]/a[1]').click()
time.sleep(2)

# juguetes
driver.find_element_by_xpath('//*[@id="anonCarousel1"]/ol/li[9]/div/span[1]').click()

# mayor descuento
dropdown=driver.find_element_by_xpath('//*[@id="sorting_dropdown0"]/span/span')
dropdown.click()

orden='Descuento: de mayor a menor'

WebDriverWait(dropdown, 5).until(EC.element_to_be_clickable((By.XPATH, '//*[contains(text(),'+'"'+orden+'"'+')]'))).click()

time.sleep(2)


In [13]:
# parseo con bs4
soup=bs(driver.page_source, 'html.parser')

# todos los productos
productos=soup.find_all('div', class_='DealGridItem-module__dealItem_2X_WLYkJ3-dM0LtXI9THcu DealGridItem-module__withBorders_2jNNLI6U1oDls7Ten3Dttl DealGridItem-module__withoutActionButton_2OI8DAanWNRCagYDL2iIqN')


# precio
precio=[p.find_all('a')[1].text for p in productos]

# oferta
oferta=[p.find_all('a')[2].text for p in productos]

# calificacion
votos=[]
for p in productos:
    try:
        votos.append(p.find_all('a')[3].attrs['aria-label'])

    except:
        votos.append('')

driver.quit()
    

In [14]:
# dataframe
columnas=['oferta', 'precio', 'calificacion']

df=pd.DataFrame(columns=columnas)

df['oferta']=oferta
df['calificacion']=votos
df['precio']=precio

df.head()

,oferta,precio,calificacion
0,"Hape Tambor para Bebé, Tambor Colorido que Rue...","Precio de la oferta: 20,90€ Precio recomendado...",Calificación: 4 estrellas y media (1705 reseñas)
1,Oferta en Famosa,"Precio de la oferta: 32,28€ Precio recomendado...",Calificación: 4 estrellas y media (352 reseñas)
2,Juguetes Bebes 1 año Bailando Caminando Pato H...,"Precio de la oferta: 22,94€ Precio recomendado...",Calificación: 4 estrellas y media (107 reseñas)
3,"Super Mario Peluche Mario/ Yoshi, 27 cm , colo...","Precio de la oferta: 15,74€ Precio recomendado...",Calificación: 4 estrellas (118 reseñas)
4,Papo Figura de Tigre de pie 50208,"Precio de la oferta: 8,00€ Precio anterior: 13...",Calificación: 4 estrellas y media (650 reseñas)
